# Machine Learning Project : LDA (Linear Discrimination Analysis)

# Introduction 

Key objectives of the project:

1. Select a supervised machine learning classification algorithm.

2. Study it in detail (theoretically and empirically).

3. Identify a data characteristic that affects its performance.

4. Propose a variant of the algorithm that addresses this weakness.

5. Evaluate the modified version on the OpenML-CC18 benchmark datasets.


----

<br></br>

In this project, we selected Linear Discriminant Analysis (LDA) and focused on the characteristic noise/outliers, which LDA is known to be sensitive to. As a classic algorithm, LDA relies heavily on accurate estimates of class means and a shared covariance matrix, making its performance vulnerable to distortions caused by extreme values in the training data.

This project investigates the robustness of standard LDA against synthetic outliers and proposes a robust variant designed to mitigate this sensitivity. The core research questions are:

- To what extent does the performance of standard LDA degrade when outliers are introduced into the training data?

- Can a variant of LDA, based on a robust covariance estimator, reduce this degradation and improve overall model
stability?


To address these questions, we conduct an empirical study comparing the standard LDA with our proposed robust variant
 across multiple datasets from the **OpenML-CC18 benchmark**. The evaluation follows a structured pipeline, including
 data preprocessing, synthetic outlier injection, and performance assessment via stratified cross-validation using Accuracy and Macro F1-score. This analysis aims to quantify the degradation of standard LDA under noise and demonstrate the effectiveness of our robust adaptation.


# Background

**Linear Discriminant Analysis (LDA)** is a classification and dimensionality reduction algorithm that allows for the
 optimal separation of classes. The goal and principle of LDA is to linearly combine the features of the data so that
 the labels
 of the datasets are best separated from each other, and the number of new features is reduced to a predefined count.

<img src="images/lda_goal.png" alt="LDA" style="width:4500px; height:350px; object-fit:contain; display:block;
margin:0 auto;" />


## LDA in scratch

The process of Linear Discriminant Analysis (LDA) can be broken down into five key steps.
<br></br>
##### **Step 1:** Compute the d-dimensional mean vectors for each of the k classes separately from the dataset.


LDA is a supervised machine learning technique, meaning we can utilize the known labels. In the first step, we
calculate the mean vectors $mean_c$ for all samples belonging to a specific class $c$. To do this, we filter the feature matrix by class label and compute the mean for each of the $d$ features. As a result, we obtain $k$ mean vectors (one for each of the $k$ classes), each with a length of $d$ (corresponding to the $d$ features).

<img src="images/feature_matrix.png" alt="LDA" style="width:950px; height:350px; object-fit:contain; display:block;
margin:0 auto;" />

$$
\text{mean}_c = \frac{1}{n_c - 1} \cdot \sum_{x \in C_c} \mathbf{X}_c = [\mu_1, \mu_2, \mu_3, \ldots, \mu_d]
$$

Where:

$ \mu_c = [\mu_1, \mu_2, \mu_3, \dots, \mu_d] $ is the $d$-dimensional mean vector

$ \mu_j $ is the mean of the $ j $-th feature for class $c$

$ n_c $ is the number of samples in class $c$

$ C_c $ - the set of all samples in class $c$

$ x $ - the sample vector



##### **Step 2:** Compute the scatter matrices (between-class scatter matrix and within-class scatter matrix).



The within-class scatter matrix measures the variation among samples within the same class. To find a subspace with optimal separability, we aim to minimize the values in this matrix. In contrast, the between-class scatter matrix measures the variation between different classes. For optimal separability, we aim to maximize the values in this matrix.

Intuitively, within-class scatter looks at how compact each class is, whereas between-class scatter examines how far apart different classes are.


<img src="images/class_scatter_matrices.png" alt="LDA" style="width:450px; height:350px; object-fit:contain;
display:block;
margin:0 auto;" />


The within-class scatter matrix S_W. It is calculated as the sum of the scatter matrices S_c for each individual class:

$$ S_w = \sum_{c=1}^k S_c  ,\  where \ \ \  S_c = \sum_{x \in C_c} (x - \text{mean}_c) \cdot (x - \text{mean}_c)^T $$


The between-class scatter matrix $S_B$ is derived from the differences between the class means $mean_c$ and the overall
mean of the entire dataset:
$$ S_B = \sum_{c=1}^k n_c \cdot (\text{mean}_c - \text{mean}) \cdot (\text{mean}_c - \text{mean})^T $$

where mean refers to the mean vector calculated over all samples, regardless of their class labels.



#### **Step 3:** Calculate the eigenvectors and eigenvalues for the ratio of $S_W$ and $S_B$.

As mentioned, for optimal class separability, we aim to maximize $S_B$ and minimize $S_W$. We can achieve both by
maximizing the ratio $S_B/S_W$. In linear algebra terms, this ratio corresponds to the scatter matrix $S_W^{-1}$ $S_B$,
which is maximized in the subspace spanned by the eigenvectors with the highest eigenvalues. The eigenvectors define the directions of this subspace, while the eigenvalues represent the magnitude of the distortion. We will select the $m$ eigenvectors associated with the highest eigenvalues.

$$
A\nu = \lambda\nu \quad \text{with} \quad A = S_W^{-1} S_B
$$

#### **Step 4:** Sort the eigenvectors in descending order of their corresponding eigenvalues, and select the $m$eigenvectors with the largest eigenvalues to form a $d × m$ - dimensional transformation matrix $W$.

The goal is not only to project the data into a subspace that enhances class separability but also to reduce
dimensionality. The eigenvectors will define the axes of our new feature subspace. To decide which eigenvectors to discard for the lower-dimensional subspace, we need to examine their corresponding eigenvalues. In simple terms, the eigenvectors with the smallest eigenvalues contribute the least to class separation, and these are the ones we want to drop. The typical approach is to rank the eigenvalues in descending order and select the top $m$ eigenvectors. $m$ is a freely chosen parameter. The larger $m$, the less information is lost during the transformation.

After sorting the eigenpairs by decreasing eigenvalues and selecting the top $m$ pairs, the next step is to construct
the $d × m$ - dimensional transformation matrix $W$. This is done by stacking the m selected eigenvectors horizontally, resulting in the matrix $W$:

$$
W =
\begin{pmatrix}
e_1^{(1)} & e_1^{(2)} & \cdots \\
e_2^{(1)} & e_2^{(2)} & \cdots \\
\vdots & \vdots & \ddots \\
e_d^{(1)} & e_d^{(2)} & \cdots
\end{pmatrix}
$$

The first column of $W$ represents the eigenvector corresponding to the highest eigenvalue, the second column represents the eigenvector corresponding to the second highest eigenvalue, and so on.



#### **Step 5:** Use W to project the samples onto the new subspace.

In the final step, we use the d × m-dimensional transformation matrix W, which we composed from the top m selected eigenvectors, to project our samples onto the new subspace:

$$Z = X⋅W,$$

where $X$ is the initial $n × d$ - dimensional feature matrix representing our samples, and $Z$ is the newly
transformed $n × m$ - dimensional feature matrix in the new subspace. This means that the selected eigenvectors serve
as the “recipes” for transforming the original features into the new features (the Linear Discriminants): The eigenvector with the highest eigenvalue provides the transformation recipe for $LD1$, the eigenvector with the second highest eigenvalue corresponds to $LD2$, and so on.


<img src="images/eug_mx.png" alt="LDA" style="width:950px; height:350px; object-fit:contain;
display:block;
margin:0 auto;" />


# Datasets Description

3. Dataset Description
3.1 Benchmark Source

We use datasets from the OpenML CC-18 Curated Classification Benchmark Suite, which contains diverse classification datasets standardized for ML evaluation.

3.2 Dataset Loading & Preprocessing

Datasets obtained in ARFF format

Loaded locally using scipy.io.arff.loadarff

Features split into:

Numerical

Categorical

Preprocessing steps:

Numerical: Standardization

Categorical: One-Hot Encoding

3.3 Dataset Summary Table

# Baseline Method: Standard LDA
4.1 Implementation Overview

Preprocess each dataset

Use stratified 5-fold cross-validation

Evaluate:

Accuracy

Macro F1-score

Evaluate both:

Clean data

Noisy data (outliers artificially injected into training folds)

4.2 Noise Injection Strategy

Explain briefly:

A percentage of training samples randomly selected

Feature values perturbed with extreme values (controlled outlier strength)

Only training folds are modified, test folds remain clean

4.3 Baseline Performance Results

(Insert your baseline results table here)

4.4 Observations

Summarize what happened when noise was added

Identify datasets most affected by outliers

# Proposed Modification

5. Proposed Method: Outlier-Filtered LDA
5.1 Motivation

LDA relies on estimating means and covariance from all data points. Outliers strongly distort these estimates.
Idea: remove samples with extreme z-scores before fitting LDA → more stable covariance → more robust classifier.

5.2 Algorithm Description

Steps:

Compute z-scores per feature

Mark samples exceeding z_thresh in ANY feature as outliers

Remove them from the training data

Fit standard LDA on filtered data

Test normally (no filtering applied to test set)

5.3 Implementation Code

(Include your FilteredLDA class and explanation)

5.4 Expected Benefits

More stable class means

Better covariance estimate

Less boundary distortion

Improved accuracy under noisy conditions

# Empirical Evaluation
6.1 Experimental Setup

Same 5-fold CV as baseline

Same noise setup

Same preprocessing pipeline

Metrics: Accuracy, Macro F1

6.2 Results Table

(Insert merged table: baseline vs filtered)

| Dataset | acc_noisy | acc_noisy_filtered | Δacc | f1_noisy | f1_noisy_filtered | Δf1 |

6.3 Visual Comparison

Insert the scatter plots:

Scatter Plot: Accuracy (Noisy Data)

(Your plot goes here)

Scatter Plot: Macro F1 (Noisy Data)

(Your plot goes here)

Optional: Bar chart for improvement per dataset

(Your bar plot if you include one)

# Results and Analysis

7. Results Analysis
7.1 Overall Improvements

Most datasets lie above the diagonal → filtered LDA performs better

Outlier-filtering improves robustness without requiring complex models

7.2 Dataset-Level Behavior

Summaries:

Strong improvements: dataset4.arff

Moderate: dataset3.arff, dataset2.arff, dataset8.arff

Neutral: dataset7.arff, dataset9.arff

Small negative: dataset10.arff (possible over-filtering)

7.3 Interpretation

Z-score filtering protects LDA from extreme values

Means + covariance become more stable

Model generalizes better under noisy training data

# Conclusions 
8. Conclusions
8.1 Summary

Baseline LDA is highly sensitive to outliers

We implemented a simple, effective outlier-filtered LDA

The variant consistently improved performance on noisy data

8.2 Strengths

Simple

Computationally cheap

Easy to interpret

Significantly improves robustness on many datasets

8.3 Limitations

May remove useful data if threshold is too strict

Effectiveness depends on dataset size and distribution

8.4 Future Work

Explore robust covariance estimation (e.g., shrinkage, MinCovDet)

Use Mahalanobis distance for more intelligent outlier detection

Extend to multiclass or high-dimensional datasets

Combine filtering with dimensionality reduction

## Appendix
A.1 Full Code Listing

(Place all helper functions, classes, etc.)

A.2 Environment & Libraries

Python version

sklearn version

pandas version